In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer

device = "cuda" # the device to load the model onto

model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [2]:
import json

json_file_path = '../../ExTES.json'  # Change 'output.json' to your desired file name

# Write the list to the JSON file
with open(json_file_path, 'rb') as json_file:
    extes_data = json.load(json_file)

In [3]:
file_path = '.././scenes.txt'  # Replace 'your_text_file.txt' with the actual path to your text file

# Open the file in read mode
with open(file_path, 'r') as file:
    # Read all lines from the file and split them based on newlines
    scenes = file.read().splitlines()

# Now, 'lines' is a list containing each line from the file
print(scenes)


['communication challenges', 'breakups or divorce', 'difficulties in a friendship', 'coping with the illness or death of a loved one', 'work-related stress and burnout', 'financial worries and uncertainty', 'academic stress', 'depression and low mood', 'managing bipolar disorder', 'anxiety and panic', 'parenthood and parenting challenges', 'low self-esteem or lack of confidence', 'body image concerns and eating disorders', 'coping with a diagnosis or medical treatment', 'addiction and recovery', 'coping with the illness of a family member', 'feeling overwhelmed and stressed', 'grief and loss', 'healing from sexual assault or violence', 'surviving and recovering from physical or emotional abuse', 'healing from abuse or domestic violence']


In [4]:
emotion_data = []
not_available = []
for i in range(len(extes_data)):
    try:
        if (extes_data[i]['scene'].strip().lower() or extes_data[i]['description'].strip().lower()) in scenes:
            
            emotion_data.append(extes_data[i])
            
    except:
        not_available.append(i)

In [5]:
def create_list(data, value):
    s = []
    for i in range(len(data)):
        #print(value.lower())
        if data[i]['scene'].lower().rstrip() == value.lower():
            s.append(data[i])
            
    return s
    
combined_scenes = []
for i in range(0,21):
    
    s = create_list(emotion_data, scenes[i])
    if len(s)>200:
        combined_scenes.append(s)
    

In [6]:
combined_scenes[0] = combined_scenes[0][0:1000]

In [7]:
#item  = combined_scenes[0][0]

formatted_data1 = []

for i in range(len(combined_scenes)):
    for item in combined_scenes[i]:
        formatted_data = []
        scene = item['scene'].lower()
        #description = item['description']
        content = item['content']
        """formatted_data.append({"scene": scene, "role": "scene"})
        if item['description']:
            formatted_data.append({"description": description, "role": "description"})"""
        
        formatted_data.append({"scene": scene, "role": "scene"})
        for element in content:
            for role, content_text in element.items():
                if role=="User":
                    role = "user"
                elif role=="AI":
                    role = "assistant"
                else:
                    continue    
                    
                formatted_data.append({"role": role, "content": content_text})
                
        formatted_data1.append(formatted_data)

In [8]:
import pandas as pd

# Define column names
columns = ['scene', 'content']

# Create an empty DataFrame with specified columns
df = pd.DataFrame(columns=columns)

# Display the empty DataFrame
print(df)

for i in range(len(formatted_data1)):
    empty_dict = {}
    empty_dict['scene'] = formatted_data1[i][0]['scene']
    #empty_dict['description'] = formatted_data1[i][1]['description']  
    empty_dict['content'] = formatted_data1[i]
    
    df = df.append(empty_dict, ignore_index=True)

Empty DataFrame
Columns: [scene, content]
Index: []


/tmp/ipykernel_631447/3212359004.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(empty_dict, ignore_index=True)
/tmp/ipykernel_631447/3212359004.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(empty_dict, ignore_index=True)
/tmp/ipykernel_631447/3212359004.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(empty_dict, ignore_index=True)
/tmp/ipykernel_631447/3212359004.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(empty_dict, ignore_index=True)
/tmp/ipykernel_631447/3212359004.py:18: FutureWarning: The frame.append method is deprecated and will be removed fro

In [9]:
df.head()

,scene,content
0,communication challenges,"[{'scene': 'communication challenges', 'role':..."
1,communication challenges,"[{'scene': 'communication challenges', 'role':..."
2,communication challenges,"[{'scene': 'communication challenges', 'role':..."
3,communication challenges,"[{'scene': 'communication challenges', 'role':..."
4,communication challenges,"[{'scene': 'communication challenges', 'role':..."


In [10]:
print(df['scene'].value_counts())

communication challenges                           1000
breakups or divorce                                 855
coping with the illness or death of a loved one     626
work-related stress and burnout                     253
academic stress                                     250
depression and low mood                             248
Name: scene, dtype: int64


In [11]:
df.dropna(inplace=True)

In [12]:
from sklearn.model_selection import StratifiedShuffleSplit
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

for train_index, test_index in sss.split(df, df['scene']):
    X_train, X_test = df.iloc[train_index], df.iloc[test_index]

In [13]:
X_train['content'] = X_train['content'].apply(lambda x: x[1:])

/tmp/ipykernel_631447/3645541154.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train['content'] = X_train['content'].apply(lambda x: x[1:])


In [14]:
X_test['content'] = X_test['content'].apply(lambda x: x[1:])

/tmp/ipykernel_631447/264133183.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test['content'] = X_test['content'].apply(lambda x: x[1:])


In [15]:
from datasets import DatasetDict
from datasets import Dataset
datasets_train_test = DatasetDict({
    "train": Dataset.from_pandas(X_train),
    "test": Dataset.from_pandas(X_test)
    })

In [16]:
import ast 
def apply_chat_template(example, tokenizer):
    messages = (example["content"])
    try:
        print("tokenized")
        tokenized_content = tokenizer.apply_chat_template(messages, tokenize=False)
        return {'content': tokenized_content}
    except Exception as e:
        print(f"Unable to tokenize:")
        text =  "None"
        return {'content': " "}

In [17]:
import re
import random
from multiprocessing import cpu_count
column_names = ['scene','content']
raw_datasets = datasets_train_test.map(apply_chat_template,
                                num_proc=cpu_count(),
                                fn_kwargs={"tokenizer": tokenizer},
                                remove_columns=column_names,
                                desc="Applying chat template",)

Applying chat template (num_proc=112):   0%|          | 0/2585 [00:00<?, ? examples/s]

tokenized
tokenized
tokenized
Unable to tokenize:
tokenized
tokenized
tokenized
tokenized
tokenized
tokenized
tokenized
Unable to tokenize:
tokenized
tokenized
tokenized
tokenized
tokenized
tokenized
tokenized
tokenized
tokenized
tokenized
tokenized
tokenized
tokenized
tokenized
tokenized
tokenized
tokenized
tokenized
tokenized
tokenized
Unable to tokenize:
tokenized
tokenized
tokenized
tokenized
tokenized
tokenized
tokenized
tokenized
tokenized
tokenized
tokenized
tokenized
tokenized
Unable to tokenize:
tokenized
tokenized
tokenized
tokenized
tokenized
tokenized
tokenized

tokenized
tokenized
tokenized
tokenized
tokenized
tokenized
tokenized
tokenized
tokenized
tokenized
tokenized
tokenized
tokenized
tokenized
tokenized
tokenized
tokenized
tokenized
tokenized
tokenized
tokenized
tokenizedtokenized
tokenizedtokenized
tokenized
tokenized
tokenized
tokenized
tokenized
tokenized
tokenized
tokenized
tokenized

tokenized
tokenized
tokenized
tokenized
tokenized
tokenized
tokenized
tokenized


Applying chat template (num_proc=112):   0%|          | 0/647 [00:00<?, ? examples/s]

tokenized
tokenized
tokenized
tokenized
tokenized
tokenized
tokenized
tokenizedtokenizedtokenized


tokenized
tokenized
tokenized
tokenized
tokenized
tokenized
tokenizedtokenized

tokenizedtokenized

tokenized
tokenizedtokenizedtokenized


tokenizedtokenizedtokenized


tokenized
tokenized
tokenized
tokenizedtokenizedtokenized


Unable to tokenize:tokenizedtokenized
tokenized

tokenizedtokenizedtokenized
tokenized



tokenizedtokenizedtokenizedtokenized



tokenizedtokenized
tokenized

tokenizedtokenized

tokenizedtokenizedtokenized


tokenizedtokenized

tokenizedtokenized
tokenizedtokenized
tokenizedtokenized

tokenized


tokenizedtokenizedtokenizedtokenized


tokenizedtokenized
tokenizedtokenizedtokenized

tokenized


tokenized
tokenized
tokenized
tokenizedtokenized
tokenizedtokenizedtokenizedtokenizedtokenized
tokenized
tokenizedtokenized




tokenized


tokenizedtokenizedtokenized
tokenized


tokenized
tokenizedtokenized
tokenized

tokenizedtokenized


tokenizedtokenizedtokenizedtok

In [18]:
raw_datasets['train'][0]

{'content': "<s>[INST] Hey there, I could really use some support right now. My partner and I recently broke up after being together for five years, and I'm feeling devastated. [/INST]Of course, I'm here for you. I'm so sorry to hear that you're going through this. Can you share more about the specific challenges you faced and how you're feeling at the moment?</s>[INST] We've been arguing a lot, and it feels like we're constantly drifting apart. It just hurts to see our relationship end like this. I feel so lost and alone. [/INST]I can understand how heartbreaking it must be for your relationship to reach this point after five years together. It's completely valid that you're feeling lost and alone right now. Breakups are never easy, and it's natural to experience a range of emotions.</s>[INST] I keep questioning if I made the right decision. Maybe we could have worked things out somehow. I'm filled with doubts and regret. [/INST]Doubts and regret are normal reactions after a breakup, 

In [19]:
train_dataset = raw_datasets["train"]
eval_dataset = raw_datasets["test"]

In [20]:
from transformers import BitsAndBytesConfig
import torch

# specify how to quantize the model
quantization_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype="bfloat16",
)
device_map = {"": torch.cuda.current_device()} if torch.cuda.is_available() else None

model_kwargs = dict(
    attn_implementation="flash_attention_2", # set this to True if your GPU supports it (Flash Attention drastically speeds up model computations)
    torch_dtype="auto",
    use_cache=False, # set to False as we're going to use gradient checkpointing
    device_map=device_map,
    quantization_config=quantization_config,
)

In [ ]:
from trl import SFTTrainer
from peft import LoraConfig
from transformers import TrainingArguments

# path where the Trainer will save its checkpoints and logs
output_dir = '/home/csgrad/sunilruf/emotion_chatbot/master/sunils_code/mistra/experiment-4'

# based on config
training_args = TrainingArguments(
    fp16=True, # specify bf16=True instead when training on GPUs that support bf16
    do_eval=True,
    evaluation_strategy="epoch",
    gradient_accumulation_steps=128,
    gradient_checkpointing=True,
    gradient_checkpointing_kwargs={"use_reentrant": False},
    learning_rate=2.0e-05,
    log_level="info",
    logging_steps=5,
    logging_strategy="steps",
    lr_scheduler_type="cosine",
    max_steps=-1,
    num_train_epochs=4,
    output_dir=output_dir,
    overwrite_output_dir=True,
    per_device_eval_batch_size=1, # originally set to 8
    per_device_train_batch_size=1, # originally set to 8
    # push_to_hub=True,
    # hub_model_id="zephyr-7b-sft-lora",
    # hub_strategy="every_save",
    # report_to="tensorboard",
    save_strategy="no",
    save_total_limit=None,
    seed=42,
    
)

# based on config
peft_config = LoraConfig(
        r=64,
        lora_alpha=16,
        lora_dropout=0.1,
        bias="none",
        task_type="CAUSAL_LM",
        target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
)



In [22]:
def preprocess_dataset(dataset):
    for sample in dataset:
        # Convert 'content' field to string
        sample['content'] = str(sample['content'])
    return dataset

# Preprocess datasets
train_dataset = preprocess_dataset(train_dataset)
eval_dataset = preprocess_dataset(eval_dataset)



In [23]:
train_dataset[0]

{'content': "<s>[INST] Hey there, I could really use some support right now. My partner and I recently broke up after being together for five years, and I'm feeling devastated. [/INST]Of course, I'm here for you. I'm so sorry to hear that you're going through this. Can you share more about the specific challenges you faced and how you're feeling at the moment?</s>[INST] We've been arguing a lot, and it feels like we're constantly drifting apart. It just hurts to see our relationship end like this. I feel so lost and alone. [/INST]I can understand how heartbreaking it must be for your relationship to reach this point after five years together. It's completely valid that you're feeling lost and alone right now. Breakups are never easy, and it's natural to experience a range of emotions.</s>[INST] I keep questioning if I made the right decision. Maybe we could have worked things out somehow. I'm filled with doubts and regret. [/INST]Doubts and regret are normal reactions after a breakup, 

In [24]:
trainer = SFTTrainer(
        model="mistralai/Mistral-7B-Instruct-v0.2",
        model_init_kwargs=model_kwargs,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        dataset_text_field="content",
        tokenizer=tokenizer,
        packing=True,  # Disable packing
        peft_config=peft_config,
        max_seq_length=1024,
    )

/home/csgrad/sunilruf/miniconda3/envs/bio3/lib/python3.11/site-packages/trl/trainer/sft_trainer.py:161: UserWarning: You passed a model_id to the SFTTrainer. This will automatically create an `AutoModelForCausalLM` or a `PeftModel` (if you passed a `peft_config`) for you.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

/home/csgrad/sunilruf/miniconda3/envs/bio3/lib/python3.11/site-packages/trl/trainer/sft_trainer.py:294: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
Using auto half precision backend


In [25]:
# Set the padding token
tokenizer.pad_token = tokenizer.eos_token

# Then, create the trainer
trainer = SFTTrainer(
        model="mistralai/Mistral-7B-Instruct-v0.2",
        model_init_kwargs=model_kwargs,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        dataset_text_field="content",
        tokenizer=tokenizer,
        packing=False,  # Disable packing
        peft_config=peft_config,
        max_seq_length=4096,
    )


/home/csgrad/sunilruf/miniconda3/envs/bio3/lib/python3.11/site-packages/trl/trainer/sft_trainer.py:161: UserWarning: You passed a model_id to the SFTTrainer. This will automatically create an `AutoModelForCausalLM` or a `PeftModel` (if you passed a `peft_config`) for you.
  warnings.warn(
loading configuration file config.json from cache at /home/csgrad/sunilruf/.cache/huggingface/hub/models--mistralai--Mistral-7B-Instruct-v0.2/snapshots/41b61a33a2483885c981aa79e0df6b32407ed873/config.json
Model config MistralConfig {
  "_name_or_path": "mistralai/Mistral-7B-Instruct-v0.2",
  "architectures": [
    "MistralForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 32768,
  "model_type": "mistral",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 8,
  "rms_norm_eps": 1e-05,
  "rope_theta": 1000

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

All model checkpoint weights were used when initializing MistralForCausalLM.

All the weights of MistralForCausalLM were initialized from the model checkpoint at mistralai/Mistral-7B-Instruct-v0.2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use MistralForCausalLM for predictions without further training.
loading configuration file generation_config.json from cache at /home/csgrad/sunilruf/.cache/huggingface/hub/models--mistralai--Mistral-7B-Instruct-v0.2/snapshots/41b61a33a2483885c981aa79e0df6b32407ed873/generation_config.json
Generate config GenerationConfig {
  "bos_token_id": 1,
  "eos_token_id": 2
}

PyTorch: setting up devices


Map:   0%|          | 0/2585 [00:00<?, ? examples/s]

Map:   0%|          | 0/647 [00:00<?, ? examples/s]

Using auto half precision backend


In [26]:
trainer.train()

***** Running training *****
  Num examples = 2,585
  Num Epochs = 10
  Instantaneous batch size per device = 1
  Training with DataParallel so batch size has been adjusted to: 4
  Total train batch size (w. parallel, distributed & accumulation) = 512
  Gradient Accumulation steps = 128
  Total optimization steps = 50
  Number of trainable parameters = 54,525,952
The input hidden states seems to be silently casted in float32, this might be related to the fact you have upcasted embedding or layer norm layers in float32. We will cast back the input in torch.bfloat16.
/home/csgrad/sunilruf/miniconda3/envs/bio3/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss
0,2.671100,2.419319
1,2.237300,2.057090


***** Running Evaluation *****
  Num examples = 647
  Batch size = 4
***** Running Evaluation *****
  Num examples = 647
  Batch size = 4


: 

In [84]:
metrics = trainer.metrics
max_train_samples = len(train_dataset)
metrics["train_samples"] = min(max_train_samples, len(train_dataset))
trainer.log_metrics("train", metrics)
trainer.save_metrics("train", metrics)
trainer.save_state()

AttributeError: 'SFTTrainer' object has no attribute 'metrics'

In [85]:
trainer.save_model('/home/csgrad/sunilruf/emotion_chatbot/master/sunils_code/mistra/experiment-3')

Saving model checkpoint to /home/csgrad/sunilruf/emotion_chatbot/master/sunils_code/mistra/experiment-3
loading configuration file config.json from cache at /home/csgrad/sunilruf/.cache/huggingface/hub/models--mistralai--Mistral-7B-Instruct-v0.2/snapshots/41b61a33a2483885c981aa79e0df6b32407ed873/config.json
Model config MistralConfig {
  "architectures": [
    "MistralForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 32768,
  "model_type": "mistral",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 8,
  "rms_norm_eps": 1e-05,
  "rope_theta": 1000000.0,
  "sliding_window": null,
  "tie_word_embeddings": false,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.38.2",
  "use_cache": true,
  "vocab_size": 32000
}

tokenizer config file saved in /home/csgrad/sunilruf/emotion_chatbo

In [86]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("/home/csgrad/sunilruf/emotion_chatbot/master/sunils_code/mistra/experiment-3")
model = AutoModelForCausalLM.from_pretrained("/home/csgrad/sunilruf/emotion_chatbot/master/sunils_code/mistra/experiment-3", load_in_4bit=True, device_map="auto")

loading file tokenizer.model
loading file tokenizer.json
loading file added_tokens.json
loading file special_tokens_map.json
loading file tokenizer_config.json
loading configuration file config.json from cache at /home/csgrad/sunilruf/.cache/huggingface/hub/models--mistralai--Mistral-7B-Instruct-v0.2/snapshots/41b61a33a2483885c981aa79e0df6b32407ed873/config.json
Model config MistralConfig {
  "_name_or_path": "mistralai/Mistral-7B-Instruct-v0.2",
  "architectures": [
    "MistralForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 32768,
  "model_type": "mistral",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 8,
  "rms_norm_eps": 1e-05,
  "rope_theta": 1000000.0,
  "sliding_window": null,
  "tie_word_embeddings": false,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.38.2",


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

All model checkpoint weights were used when initializing MistralForCausalLM.

All the weights of MistralForCausalLM were initialized from the model checkpoint at mistralai/Mistral-7B-Instruct-v0.2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use MistralForCausalLM for predictions without further training.
loading configuration file generation_config.json from cache at /home/csgrad/sunilruf/.cache/huggingface/hub/models--mistralai--Mistral-7B-Instruct-v0.2/snapshots/41b61a33a2483885c981aa79e0df6b32407ed873/generation_config.json
Generate config GenerationConfig {
  "bos_token_id": 1,
  "eos_token_id": 2
}



In [98]:
messages = [ {
  'role':'user',
  'content': 'Hey, are you free? I need someone to talk to.',
},
        {
  'role':'assistant',
  'content': "Of course, I'm here to help. Feel free to share what's on your mind and I'll listen and try to provide a supportive and understanding response. What's been going on?",
  
        },
        {
  'role':'user',
  'content': "I've been feeling really down lately. I'm struggling with my mental health and it's been hard to cope. I feel overwhelmed and alone.",
        }   ]

In [99]:
input_ids = tokenizer.apply_chat_template(messages, truncation=True, add_generation_prompt=True, return_tensors="pt").to("cuda")

# inference
outputs = model.generate(
        input_ids=input_ids,
        max_new_tokens=256,
        do_sample=True,
        temperature=0.1,
        top_k=50,
        top_p=0.95
)
print(tokenizer.batch_decode(outputs, skip_special_tokens=True)[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[INST] Hey, are you free? I need someone to talk to. [/INST]Of course, I'm here to help. Feel free to share what's on your mind and I'll listen and try to provide a supportive and understanding response. What's been going on? [INST] I've been feeling really down lately. I'm struggling with my mental health and it's been hard to cope. I feel overwhelmed and alone. [/INST] I'm really sorry to hear that you've been feeling this way. It can be incredibly difficult to deal with mental health struggles, and it's important to remember that you're not alone. I'm here to listen and offer support in any way that I can.

It's okay to feel overwhelmed and it's okay to take things one day at a time. It can be helpful to focus on small things that bring you joy or make you feel better, even if they seem insignificant. And don't hesitate to reach out to friends, family, or professionals for help if you need it.

Remember that it's okay to take care of yourself and prioritize your mental health. It ca

In [100]:
input_ids = tokenizer.apply_chat_template(messages, truncation=True, add_generation_prompt=True, return_tensors="pt").to("cuda")

# inference
outputs = model.generate(
        input_ids=input_ids,
        max_new_tokens=256,
        do_sample=True,
        temperature=0.1,
        top_k=50,
        top_p=0.95
)
print(tokenizer.batch_decode(outputs, skip_special_tokens=True)[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[INST] Hey, are you free? I need someone to talk to. [/INST]Of course, I'm here to help. Feel free to share what's on your mind and I'll listen and try to provide a supportive and understanding response. What's been going on? [INST] I've been feeling really down lately. I'm struggling with my mental health and it's been hard to cope. I feel overwhelmed and alone. [/INST] I'm really sorry to hear that you've been feeling this way lately. It's important to remember that it's okay to have difficult emotions and that you're not alone in experiencing them. Mental health struggles can be incredibly challenging, but there are resources and support available to help.

Have you considered reaching out to a mental health professional, such as a therapist or counselor, to talk about what you're going through? They can provide you with tools and techniques to help manage your emotions and improve your overall well-being.

Additionally, there are many online resources and support groups that can pr

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

device = "cpu" # the device to load the model onto

model1 = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2", load_in_4bit=True, device_map="auto")
tokenizer1 = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")

In [107]:
input_ids = tokenizer1.apply_chat_template(messages, truncation=True, add_generation_prompt=True, return_tensors="pt").to("cuda")

# inference
outputs = model1.generate(
        input_ids=input_ids,
        max_new_tokens=256,
        do_sample=True,
        temperature=0.1,
        top_k=50,
        top_p=0.95
)
print(tokenizer1.batch_decode(outputs, skip_special_tokens=True)[0])

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[INST] Hey, are you free? I need someone to talk to. [/INST]Of course, I'm here to help. Feel free to share what's on your mind and I'll listen and try to provide a supportive and understanding response. What's been going on? [INST] I've been feeling really down lately. I'm struggling with my mental health and it's been hard to cope. I feel overwhelmed and alone. [/INST] I'm really sorry to hear that you've been feeling this way lately. It's important to remember that it's okay to have difficult emotions and that you're not alone in experiencing them. Mental health struggles can be incredibly challenging, but there are resources and support available to help.

Have you considered reaching out to a mental health professional, such as a therapist or counselor, to talk about what you're going through? They can provide you with tools and techniques to help manage your emotions and improve your overall well-being.

Additionally, there are many online resources and support groups that can pr